In [1]:
!which python

In [2]:
# Cell 1: Imports and Setup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from IPython.display import Image, display
import time

In [3]:
# Cell 2: Helper Functions
def setup_driver():
    """Initialize Chrome driver with debugging options"""
    chrome_options = Options()
    # chrome_options.add_argument('--headless')  # Uncomment to run headless
    chrome_options.add_argument('--verbose')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--log-level=3')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--window-size=1920,1080')
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
    chrome_options.add_argument(f'user-agent={user_agent}')
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=chrome_options)
    return driver

def take_screenshot(driver, name="debug_screenshot"):
    """Take and display a screenshot in the notebook"""
    filename = f"{name}.png"
    driver.save_screenshot(filename)
    display(Image(filename))

def highlight_element(driver, element, duration=2):
    """Highlight an element temporarily"""
    original_style = element.get_attribute('style')
    driver.execute_script(
        "arguments[0].setAttribute('style', 'border: 3px solid red; background: yellow;');",
        element
    )
    time.sleep(duration)
    driver.execute_script(
        "arguments[0].setAttribute('style', arguments[1]);",
        element, original_style
    )

In [4]:
# Cell 3: Debug Function for Jellycat
def debug_jellycat_page():
    driver = setup_driver()
    print("Starting browser...")
    
    try:
        # Navigate to page
        url = "https://www.jellycat.com/jollipop-cat/"
        driver.get(url)
        print(f"Loaded URL: {url}")
        
        # Take initial screenshot
        print("\nInitial page state:")
        take_screenshot(driver, "initial_state")
        
        # Find the stock status button
        print("\nLooking for stock button...")
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.CSS_SELECTOR,
                'input[type="submit"].button.button--primary'
            ))
        )
        
        # Highlight and show the button
        print("Found button! Highlighting...")
        highlight_element(driver, button)
        take_screenshot(driver, "button_highlighted")
        
        # Print button details
        print("\nButton details:")
        print(f"Value: {button.get_attribute('value')}")
        print(f"Class: {button.get_attribute('class')}")
        print(f"Enabled: {button.is_enabled()}")
        print(f"Displayed: {button.is_displayed()}")
        
        return driver, button
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        take_screenshot(driver, "error_state")
        driver.quit()
        raise


In [5]:
# Cell 4: Interactive Testing Functions
def try_selector(driver, selector, by=By.CSS_SELECTOR):
    """Test a selector and print results"""
    print(f"\nTrying selector: {selector}")
    try:
        elements = driver.find_elements(by, selector)
        print(f"Found {len(elements)} elements")
        
        for i, elem in enumerate(elements, 1):
            print(f"\nElement {i}:")
            print(f"  Tag: {elem.tag_name}")
            print(f"  Text: {elem.text}")
            print(f"  Value: {elem.get_attribute('value')}")
            print(f"  Visible: {elem.is_displayed()}")
            highlight_element(driver, elem, 1)
            
    except Exception as e:
        print(f"Error: {str(e)}")

def check_stock_status(button):
    """Analyze the button to determine stock status"""
    value = button.get_attribute('value').lower()
    is_disabled = not button.is_enabled()
    
    print("\nStock Status Analysis:")
    print(f"Button value: {value}")
    print(f"Button disabled: {is_disabled}")
    
    if 'out of stock' in value or is_disabled:
        print("Status: OUT OF STOCK")
    else:
        print("Status: IN STOCK")



In [8]:
# Cell 5: Run the debug session
# Execute this cell to start debugging
driver, button = debug_jellycat_page()


In [7]:

# Cell 6: Interactive testing
# Run these commands as needed
selectors_to_try = [
    'input[type="submit"]',
    '.button--primary',
    'input.button.button--primary',
    'input[value*="Stock"]'
]

for selector in selectors_to_try:
    try_selector(driver, selector)
    time.sleep(1)  # Pause between tests

check_stock_status(button)

# Keep browser open for inspection
# Run driver.quit() when done